In [67]:
from collections import Counter
import string
import re

In [2]:
with open('G:\python_github_dump\github_python_dump000000000001', encoding='utf8') as f:
    dump = f.read()

In [3]:
dump = dump.replace('",false,1', '')
splitted = re.split(r'[a-z0-9]{40},\d+,"', dump)
splitted = splitted[1:]

In [4]:
def parse_imports(imports):
    '''Returns list of tuples containing import name and the name to search for'''
    imports_parsed = set()
    for import_line in imports:
        import_v_from = re.split(r' +', import_line)[0]
        if import_v_from == 'import':
            libs = re.split(r' ?, ?', import_line.replace('import', '').strip())
            for l in libs:
                if ' as ' in l:
                    lib_name = re.split(r' +as +', l)[0].strip()
                    lib_alias = re.split(r' +as +', l)[1].strip()
                    imports_parsed.add((lib_name, lib_alias))
                else:
                    imports_parsed.add((l.strip(), l.strip()))
        elif import_v_from == 'from':
            lib_name = import_line.split('import')[0].replace('from ', '').strip()
            methods_objects = re.split(r' ?, ?', import_line.split('import')[-1].strip())
            for mo in methods_objects:
                if '*' == mo:
                    continue
                if ' as ' in mo:
                    mo_name = re.split(r' +as +', mo)[0].strip()
                    mo_alias = re.split(r' +as +', mo)[1].strip()
                    imports_parsed.add((f'{lib_name}.{mo_name}', mo_alias))
                else:
                    imports_parsed.add((f'{lib_name}.{mo.strip()}', mo.strip()))
    return imports_parsed

In [8]:
def find_objects(input_string, imports_list):
    '''
    Finds variable names for created objects
    '''
    objects = set()
    for imp in imports_list:
        for f in re.findall(r'\w+ *= *{}'.format(imp[1]), input_string):  # TODO: pozbyc sie argumentow
            objects.add((f'{imp[0]}___derivative', f.split('=')[0].strip()))
    return objects

In [125]:
def remove_parentheses_brackets(input_string):
    '''
    Removes parentheses and/or brackets with what is inside them
    '''  # TODO: test roznych zagniezdzonych
    parens = []
    opening_pb = []
    for index, ch in enumerate(input_string):
        if ch == '(' or ch == '[':
            opening_pb.append(index)
        elif (ch == ')' or ch == ']') and opening_pb:
            op = opening_pb.pop()
            if len(opening_pb) == 0:
                parens.append(input_string[op:index+1])
    for p in parens:
        input_string = input_string.replace(p, '')
    input_string = re.sub(r'[\(\)\[\]]', '', input_string).strip()
    return input_string
    

In [7]:
def remove_comments_docstrings(input_string):
    input_string = re.sub(r'#.*\n', '\n', input_string)
    start = None
    for match in re.finditer(r'"""+|\'\'\'+', input_string):
        if start is None:
            start = match.span()[0]
        else:
            to_replace = input_string[start: match.span()[1]]
            input_string = input_string.replace(to_replace, '')
            start = None
    return input_string

In [6]:
def balanced_parentheses(input_string):
    return input_string.count('(') == input_string.count(')')

In [9]:
def extract_inheritance(input_string, imports_list):
    occurences_dict = {}
    for imp in imports_list:
        for occ in re.findall(r'class .*\({}\):'.format(imp[1]), input_string):
            if imp[0] not in occurences_dict:
                occurences_dict[imp[0]] = 1
            else:
                occurences_dict[imp[0]] += 1
    return occurences_dict

In [121]:
def extract_function_attribute(input_string, func_name):
    occurences_dict = {}
    for match in re.finditer(r'(\W|^){}(\.|\(|\[|\'|\"| )'.format(func_name[1]), input_string):
        start = match.span()[0] + re.search(func_name[1], match.group()).span()[0]
        end = match.span()[1]
        if '=' in input_string[end:end+2]:
            continue
        for i in range(len(input_string[:start]), len(input_string)):
            if (input_string[i] not in list(string.ascii_letters) + ['.', '(', ')', '_'] and balanced_parentheses(input_string[start:i+1])) or input_string[i] == '\n':
                print('LINE:   ', input_string[start:start+100].replace('\n', ' '))
                dict_key = f"{func_name[0]}{remove_parentheses_brackets(input_string[start:i].strip(':'))[len(func_name[1]):]}"
                if dict_key not in occurences_dict:
                    occurences_dict[dict_key] = 1
                    print('ADDED:  ', dict_key)
                else:
                    occurences_dict[dict_key] += 1
                    print('ADDED:  ', dict_key)
                break
    return occurences_dict

In [143]:
snippet = splitted[30]
snippet = remove_comments_docstrings(snippet)
snippet = re.sub(r'def .+\n', ' ', snippet)
snippet = snippet.replace('\\\n', ' ')
import_lines = []
for import_line in re.findall(r' *import .+\n| *from .+ import .+\n', snippet):
    import_lines.append(import_line.strip())
    snippet = snippet.replace(import_line, ' ')

occurences_dict = {}  
imports = parse_imports(import_lines)
imports.update(find_objects(snippet, imports))
for imp in imports:
    occurences_dict = dict(Counter(occurences_dict) + Counter(extract_function_attribute(snippet, imp)))
print(occurences_dict)

LINE:    writer.writerow(['rivid', 'lat', 'lon', 'z'])         for feature in ogr_drainage_line_shapefile_lyr
ADDED:   csv.writer___derivative.writerow
LINE:    writer.writerow([                 feature.GetField(river_id),                 centroid_pt[1],       
ADDED:   csv.writer___derivative.writerow
LINE:    osr_geographic_proj.ImportFromEPSG(4326)     proj_transform = None     if ogr_drainage_line_shapefil
ADDED:   osgeo.osr___derivative.ImportFromEPSG
LINE:    osr.SpatialReference()     osr_geographic_proj.ImportFromEPSG(4326)     proj_transform = None     if
ADDED:   osgeo.osr.SpatialReference
LINE:    osr.CoordinateTransformation(             ogr_drainage_line_shapefile_lyr_proj, osr_geographic_proj)
ADDED:   osgeo.osr.CoordinateTransformation
LINE:    csv_writer(outfile)         writer.writerow(['rivid', 'lat', 'lon', 'z'])         for feature in ogr
ADDED:   csv.writer
LINE:    open_csv(out_csv_file, 'w') as outfile:         writer = csv_writer(outfile)         writer.writerow

In [144]:
imports

{('..helper_functions.open_csv', 'open_csv'),
 ('..open_shapefile', 'open_shapefile'),
 ('csv.writer', 'csv_writer'),
 ('csv.writer___derivative', 'writer'),
 ('osgeo.gdal', 'gdal'),
 ('osgeo.osr', 'osr'),
 ('osgeo.osr___derivative', 'osr_geographic_proj'),
 ('osgeo.osr___derivative', 'proj_transform')}